In [2]:
import requests 
import yaml
import json
import datetime as dt
import time
import os
import random
from azure.storage.blob import BlobServiceClient, BlobType
from email_validator import validate_email, EmailNotValidError
from funcs.utils import log_azure, request, load_json, clean_field_text, has_valid_email, load_config
from funcs.funcs import combine_qa_keys, get_sm_survey_responses

with open("api-key.yaml", "r") as file:
    data = yaml.full_load(file)

# SurveyMonkey Survey
SM_DATA = data['sm']['real']
# CareerOneStop Survey 
COS_DATA = data['cos']


---

##### **Main Functions** 

* `get_qa_key()` - GET (or load cached copy) of question/answer key from SM or COS 

<br>

* `combine_qa_keys()` - Combine the SM and COS question/answer keys into one combined key/translation map between the APIs
    - Generates a refreshed map if a change is detected in the SM survey or the COS survey. 
        - The COS survey should not change at all. 
        - Changes in the SM survey should not affect its ability to match the answer keys in the COS survey.

<br>

* `get_sm_responses()` - GET SM survey responses 

<br>

* `process_sm_responses()` - filter and process new SM responses from get_sm_responses()
    - Checks against DB for already processed responses 
    - Checks if response includes valid email address (`has_valid_email()`)
    - Checks for unexpected response ids vs. the combined Q/A key.
    - Loads new responses into database (into 'processing' table) until they are finished

<br>

* `translate_post_cos()` - Use combined answer to translate the processed SM responses to COS JSON format
    - Loads responses from processing table in
    - Adds matching COS information to response questions/answers 
    - If response is missing a required skills-survey answer, fills with "beginner" 

<br>

* `compose_email()` - Extract list of recommended jobs from COS response and compose an email.

In [3]:
sm_survey_responses = get_sm_survey_responses()

INFO: GET {'https://api.surveymonkey.com/v3/surveys/513506444/responses/bulk'} -- {200} -- 3.17 -- {datetime.datetime(2023, 10, 11, 11, 0, 45, 515105)}


In [4]:
sm_survey_responses

{'data': [{'id': '118440150887',
   'recipient_id': '',
   'collection_mode': 'default',
   'response_status': 'completed',
   'custom_value': '',
   'first_name': '',
   'last_name': '',
   'email_address': '',
   'ip_address': '73.165.183.94',
   'logic_path': {},
   'metadata': {'contact': {}},
   'page_path': [],
   'collector_id': '452261040',
   'survey_id': '513506444',
   'custom_variables': {},
   'edit_url': 'https://www.surveymonkey.com/r/?sm=MSXCvuilN9Vf0ZobVXFjPswyoNLgo_2FfEjRoZb0vM7grlb8CTskkkTNj66m1GwMlq',
   'analyze_url': 'https://www.surveymonkey.com/analyze/browse/8xd9D8KA37YeLpeTvx8j2O9lfNbj2kqrYZtRcHvmtdM_3D?respondent_id=118440150887',
   'total_time': 1043,
   'date_modified': '2023-10-10T23:03:01+00:00',
   'date_created': '2023-10-10T22:45:38+00:00',
   'href': 'https://api.surveymonkey.com/v3/surveys/513506444/responses/118440150887',
   'pages': [{'id': '43673968',
     'questions': [{'id': '144588883',
       'answers': [{'choice_id': '1070603277'}]},
      

In [39]:
combined_map = combine_qa_keys(fetch=False)    
skills_matcher_ids = set(combined_map['skills-matcher'].keys())
non_skills_matcher_ids = set(combined_map['non-skills-matcher'].keys())


processed_responses = []
placeholder_processed_response_ids = []
for resp in sm_survey_responses['data']: 
    if resp['id'] not in placeholder_processed_response_ids: # and has_valid_email(resp, check_deliverability=False) 
        # TO-DO: Do the email validation in the actual email sending function 

        resp_dict = {
        'response_id':resp['id'],
        'collector_id':resp['collector_id'], 
        'questions':[] 
        }

        ## Add questions information
        for p in resp['pages']:
            for q in p['questions']:

                # Match q to key from combined_map, based on question type and sm question_id
                question_type = 'non-skills-matcher' if q['id'] in non_skills_matcher_ids else 'skills-matcher'
                q_map = combined_map[question_type][q['id']] 

                # Match given answers in q['answers'] to answers in q_map['answers']
                if q_map['answers'] is not None:
                    # e.g. of q objects: {'id': '144588883', 'answers': [{'choice_id': '1070603278'}]}
                    q_map_answer_key = {a['id']['sm']:a for a in q_map['answers']}
                    answers = [q_map_answer_key[a['choice_id']] for a in q['answers'] if 'choice_id' in a.keys()]
                else:
                    # e.g. {'id': '143922396', 'answers': [{'tag_data': [], 'text': '19977'}]}
                    answers = q['answers']

                resp_dict['questions'].append({'question_id':q_map['question_id'], 
                                               'page_number':q_map['page_number'],
                                               'question_number':q_map['question_number'], 
                                               'question_family': q_map['question_family'],
                                               'question_text':q_map['question_text'],
                                               'question_type':question_type, 
                                               'answers':answers})
                                            
        processed_responses.append(resp_dict)



## -- 3. Write (raw) new responses to processing table in DB (TO-DO)  -- ## 
# load_to_db(...)
## -------------------------------------------------------------------- ## 
# [q for q in processed_responses[0]['questions'] if q['question_type'] == 'non-skill-matcher'] 

processed_responses

/Users/alexsocarras/tech_impact/career-onestop/funcs/utils.py:95: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


[{'response_id': '118440150887',
  'collector_id': '452261040',
  'questions': [{'question_id': {'sm': '144588883'},
    'page_number': 1,
    'question_number': {'sm': 1},
    'question_family': 'single_choice',
    'question_text': {'sm': 'How did you learn about the survey?'},
    'question_type': 'non-skills-matcher',
    'answers': [{'id': {'sm': '1070603277'}, 'text': {'sm': 'Michael'}}]},
   {'question_id': {'sm': '150376558'},
    'page_number': 1,
    'question_number': {'sm': 2},
    'question_family': 'single_choice',
    'question_text': {'sm': 'Are you taking this survey online or in person?'},
    'question_type': 'non-skills-matcher',
    'answers': [{'id': {'sm': '1108449077'}, 'text': {'sm': 'Online'}}]},
   {'question_id': {'sm': '143922396'},
    'page_number': 1,
    'question_number': {'sm': 3},
    'question_family': 'open_ended',
    'question_text': {'sm': 'What zip code do you currently live in?'},
    'question_type': 'non-skills-matcher',
    'answers': [{'ta

In [32]:
display(q)
display(q_map)

{'id': '145869785',
 'answers': [{'tag_data': [], 'text': 'katieschroderrs@gmail.com'}]}

{'question_id': {'sm': '145869785'},
 'page_number': 22,
 'question_number': {'sm': 82},
 'question_family': 'open_ended',
 'question_text': {'sm': 'Email Address:'},
 'answers': None}

In [22]:
combined_map['non-skills-matcher']

{'144588883': {'question_id': {'sm': '144588883'},
  'page_number': 1,
  'question_number': {'sm': 1},
  'question_family': 'single_choice',
  'question_text': {'sm': 'How did you learn about the survey?'},
  'answers': [{'id': {'sm': '1070603277'}, 'text': {'sm': 'Michael'}},
   {'id': {'sm': '1070603278'}, 'text': {'sm': 'Kim'}},
   {'id': {'sm': '1070603279'}, 'text': {'sm': "Ci'Aira"}},
   {'id': {'sm': '1070603280'}, 'text': {'sm': "Sade'"}},
   {'id': {'sm': '1070603281'}, 'text': {'sm': 'Jabrielle'}},
   {'id': {'sm': '1076832322'}, 'text': {'sm': 'Other (please specify)'}}]},
 '150376558': {'question_id': {'sm': '150376558'},
  'page_number': 1,
  'question_number': {'sm': 2},
  'question_family': 'single_choice',
  'question_text': {'sm': 'Are you taking this survey online or in person?'},
  'answers': [{'id': {'sm': '1108449076'}, 'text': {'sm': 'In person'}},
   {'id': {'sm': '1108449077'}, 'text': {'sm': 'Online'}}]},
 '143922396': {'question_id': {'sm': '143922396'},
  'pa

In [30]:
# answers = [{'sm':a['choice_id'],
#             'cos':q_map['answers']} for a in q['answers'] if ]

answer_key = {a['id']['sm']:a  for a in q_map['answers']}
answer_key

{'1066658702': {'id': {'sm': '1066658702', 'cos': 1.534},
  'text': {'sm': 'Beginner (Complete a timesheet)',
   'cos': 'Complete a timesheet'}},
 '1066658703': {'id': {'sm': '1066658703', 'cos': 2.4145},
  'text': {'sm': 'Basic', 'cos': ''}},
 '1066658704': {'id': {'sm': '1066658704', 'cos': 3.295},
  'text': {'sm': 'Skilled (Monitor project progress to complete it on time)',
   'cos': 'Monitor project progress to complete it on time'}},
 '1066658705': {'id': {'sm': '1066658705', 'cos': 4.1755},
  'text': {'sm': 'Advanced', 'cos': ''}},
 '1066658706': {'id': {'sm': '1066658706', 'cos': 5.056},
  'text': {'sm': 'Expert (Manage a $10m company)',
   'cos': 'Manage a $10m company'}}}

In [28]:
{a['id']['sm']:a['id']['cos'] for a in q_map['answers']}

{'1066658702': 1.534,
 '1066658703': 2.4145,
 '1066658704': 3.295,
 '1066658705': 4.1755,
 '1066658706': 5.056}

In [15]:
processed_responses

[]

In [11]:
combined_map = combine_qa_keys()

/Users/alexsocarras/tech_impact/career-onestop/funcs/utils.py:95: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [13]:
combined_map['skills-matcher']

{'143974309': {'question_id': {'sm': '143974309', 'cos': '2.C.1.a'},
  'page_number': 21,
  'question_number': {'sm': 42, 'cos': 1},
  'question_family': 'single_choice',
  'question_text': {'sm': 'How much do you know about business planning and leadership?',
   'cos': 'How much do you know about business planning and leadership?'},
  'answers': [{'id': {'sm': '1066658702', 'cos': 1.534},
    'text': {'sm': 'Beginner (Complete a timesheet)',
     'cos': 'Complete a timesheet'}},
   {'id': {'sm': '1066658703', 'cos': 2.4145},
    'text': {'sm': 'Basic', 'cos': ''}},
   {'id': {'sm': '1066658704', 'cos': 3.295},
    'text': {'sm': 'Skilled (Monitor project progress to complete it on time)',
     'cos': 'Monitor project progress to complete it on time'}},
   {'id': {'sm': '1066658705', 'cos': 4.1755},
    'text': {'sm': 'Advanced', 'cos': ''}},
   {'id': {'sm': '1066658706', 'cos': 5.056},
    'text': {'sm': 'Expert (Manage a $10m company)',
     'cos': 'Manage a $10m company'}}]},
 '143

In [ ]:
sm_responses